# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [42]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [44]:
weather_data = pd.read_csv('Weather_Data.csv')

weather_data.head()

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Windspeed
0,0,Rikitea,-23.1203,-134.9692,297.23,76,5,5.82
1,1,Mataura,-46.1927,168.8643,281.77,97,100,0.60
2,2,Cidreira,-30.1811,-50.2056,291.18,57,36,3.20
3,3,Busselton,-33.6500,115.3333,284.29,79,18,3.40
4,4,Benguela,-12.5763,13.4055,297.00,76,5,3.71


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [25]:
gmaps.configure(g_key)

In [26]:
location = weather_data[["Latitude", "Longitude"]]
humid = weather_data['Humidity']

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(location, weights=humid,
                                dissipating=False, max_intensity = 100, point_radius=1)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [35]:
perfect_weather = weather_data.loc[(weather_data['Humidity'] < 35) & (weather_data['Temperature'] > 275)
                                   & (weather_data['Windspeed'] < 2), :]

perfect_weather = perfect_weather.dropna(how='any')
perfect_weather

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Windspeed
25,25,Yangi Marg`ilon,40.4272,71.7189,289.22,26,99,0.72
28,28,São Miguel do Araguaia,-13.2750,-50.1628,309.23,30,63,1.96
51,51,Umm Hājar,13.2954,19.6966,306.71,29,25,0.73
76,76,Price,39.5994,-110.8107,283.56,30,0,1.98
98,98,Marzuq,14.4000,46.4667,300.04,27,99,0.11
102,102,Terra Santa,-2.1042,-56.4869,310.72,31,48,1.70
139,139,Dārāb,28.7519,54.5444,297.71,16,0,1.46
211,211,Kulachi,31.9286,70.4592,301.07,12,0,0.75
222,222,Santa Rosalía,27.3167,-112.2833,299.82,34,99,0.15
268,268,Dera Ghazi Khan,30.0561,70.6348,300.49,13,0,1.65


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [49]:
hotel_name = []
location = perfect_weather[["Latitude", "Longitude"]]

for i in location:
    Lat = perfect_weather["Latitude"]
    Long = perfect_weather["Longitude"]
    
    # geocoordinates
    target_coordinates = [Lat], [Long]
    target_radius = 5000
    target_type = "hotel"

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params).json()
    
    #hotel_name.append(response['results'][0]['name'])

response
#hotel_df = perfect_weather.insert(1, "Hotel Name", hotel_name)
#hotel_df

{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}

,Latitude,Longitude
0,-23.1203,-134.9692
1,-46.1927,168.8643
2,-30.1811,-50.2056
3,-33.6500,115.3333
4,-12.5763,13.4055
...,...,...
547,46.3834,-75.9660
548,34.5023,-97.9578
549,19.8000,85.8500
550,42.0418,-70.6723


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
